In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

In [223]:
google_app_data = pd.read_csv(r'''C:\Users\Akshay\Desktop\MIS\First_Sem\Data_Sci_Eng_Mthd\Assignment3\googleplaystore.csv''',encoding="ISO-8859-1")
google_app_user_review = pd.read_csv(r'''C:\Users\Akshay\Desktop\MIS\First_Sem\Data_Sci_Eng_Mthd\Assignment3\googleplaystore_user_reviews.csv''',encoding="ISO-8859-1")

In [224]:
google_app_data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
google_app_data.drop(google_app_data.loc[google_app_data['Type'] == "0"].index, inplace=True, axis=0)
google_app_data.drop(google_app_data.loc[google_app_data['Installs'] == "0"].index, axis=0, inplace=True)
google_app_data.drop(google_app_data.loc[google_app_data['Rating'].isna()].index, axis=0, inplace=True)
google_app_data.drop(google_app_data.loc[google_app_data['Size'] == "Varies with device"].index, axis=0, inplace=True)
google_app_data.info()

In [ ]:
google_app_data['Size'] = google_app_data['Size'].apply(lambda x:float(x.replace("M", ""))*1024 if "M" in x else float(x.replace("k","")))
google_app_data['Reviews'] = google_app_data['Reviews'].astype(float)
google_app_data.reset_index(drop=True, inplace=True)
google_app_data.info()

In [226]:
values = {'Rating': 0.0, 'Type': 'Free', 'Content Rating': 'Unrated','Price':'0.0'}
google_app_data.fillna(value=values, inplace=True)
google_app_data.info()

(7729, 13)


In [228]:
def price_quality(temp):
    if temp == "0":
        return 0
    else:
        temp = temp[1:]
        temp = float(temp)
        return temp
    
    
google_app_data['Price'] = google_app_data['Price'].map(price_quality).astype(float)  
google_app_data.drop_duplicates(subset='App', keep='first', inplace=True)

In [229]:
google_app_data.drop(columns=['App', 'Last Updated', 'Current Ver', 'Android Ver'], axis=1, inplace=True)
google_app_data_with_categories = pd.DataFrame(pd.get_dummies(google_app_data))
google_app_data_with_categories.info()
google_app_data.info()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,ART_AND_DESIGN,4.1,159.0,19456.0,"10,000+",Free,0.0,Everyone,Art & Design
1,ART_AND_DESIGN,3.9,967.0,14336.0,"500,000+",Free,0.0,Everyone,Art & Design;Pretend Play
2,ART_AND_DESIGN,4.7,87510.0,8908.8,"5,000,000+",Free,0.0,Everyone,Art & Design
3,ART_AND_DESIGN,4.5,215644.0,25600.0,"50,000,000+",Free,0.0,Teen,Art & Design
4,ART_AND_DESIGN,4.3,967.0,2867.2,"100,000+",Free,0.0,Everyone,Art & Design;Creativity


In [232]:
categories_data = google_app_data['Category'].value_counts().reset_index()
categories_data.head()
plt.figure(figsize=(10,10))
pie_plot = plt.subplot(1,1,1)
plt.pie(x=categories_data['Category'], labels=categories_data['index'], autopct='%1.1f%%')
plt.legend()
pie_plot.legend(bbox_to_anchor=(1.4,1))
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.distplot(google_app_data['Rating'])
plt.legend(['Rating'])
plt.show()

Linear Regression Model 1: This model include only continueous predictors in order to predict the rating of the applicaiton.

In [236]:
google_app_data.head()
model1_data = google_app_data.drop(columns=['Rating','Category', 'Genres', 'Content Rating', 'Installs', 'Type','Size'])
Y = google_app_data.Rating
x1_train, x1_test, y1_train, y1_test = train_test_split(model1_data, Y, test_size=0.3)
linear_model = LinearRegression()
linear_model.fit(x1_train,y1_train)
linear_model.predict(x1_test)

model1_ols = sm.OLS(y1_train, x1_train[['Reviews','Price']])
model1_metrics = model1_ols.fit()
print(model1_metrics.summary())
x1_train.info()

                            OLS Regression Results                            
Dep. Variable:                 Rating   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     60.56
Date:                Sat, 17 Nov 2018   Prob (F-statistic):           1.04e-26
Time:                        17:30:35   Log-Likelihood:                -13984.
No. Observations:                4920   AIC:                         2.797e+04
Df Residuals:                    4918   BIC:                         2.798e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reviews     5.417e-07   5.34e-08     10.139      0.0